## modelo de recomendacion

In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
import re


In [5]:
df = pd.read_csv(r'D:\Denise_Estudio\henry\PI\datos\movies_data.csv')

In [6]:
# Definir la expresión regular para caracteres permitidos
regex = re.compile(r'^[a-zA-Z0-9\s\.,\?!-]+$')

# Filtrar el DataFrame
df_clean = df[df['title'].apply(lambda x: bool(regex.match(x)))]


In [7]:
df_clean

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,...,vote_count,genres_clean,companies_names,spoken_languages_clear,production_countries_clean,name_collection,return,release_year,cast_clean,director
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,...,5415.0,"Animation, Comedy, Family",Pixar Animation Studios,English,United States of America,Toy Story Collection,12.451801,1995,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Released,Jumanji,...,2413.0,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...","English, Français",United States of America,0,4.043035,1995,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0,101.0,Released,Grumpier Old Men,...,92.0,"Romance, Comedy","Warner Bros., Lancaster Gate",English,United States of America,Grumpy Old Men Collection,0.000000,1995,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156,127.0,Released,Waiting to Exhale,...,34.0,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,English,United States of America,0,5.090760,1995,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911,106.0,Released,Father of the Bride Part II,...,173.0,Comedy,"Sandollar Productions, Touchstone Pictures",English,United States of America,Father of the Bride Collection,0.000000,1995,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44395,0.0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0,104.0,Released,Robin Hood,...,26.0,"Drama, Action, Romance","Westdeutscher Rundfunk (WDR), Working Title Fi...",English,"Canada, Germany, United Kingdom, United States...",0,0.000000,1991,"Patrick Bergin, Uma Thurman, David Morrissey, ...",John Irvin
44396,0.0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0,360.0,Released,Century of Birthing,...,3.0,Drama,Sine Olivia,NaN,Philippines,0,0.000000,2011,"Angel Aquino, Perry Dizon, Hazel Orencio, Joel...",Lav Diaz
44397,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0,90.0,Released,Betrayal,...,6.0,"Action, Drama, Thriller",American World Pictures,English,United States of America,0,0.000000,2003,"Erika Eleniak, Adam Baldwin, Julie du Page, Ja...",Mark L. Lester
44398,0.0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0,87.0,Released,Satan Triumphant,...,0.0,no register,Yermoliev,NaN,Russia,0,0.000000,1917,"Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav...",Yakov Protazanov


In [8]:
valid_columns = ['id', 'title','genres_clean']
df_final = df_clean[valid_columns]

In [9]:
muestra = df_final.head(20000)


In [10]:
tfidf = TfidfVectorizer(stop_words='english')
muestra = muestra.fillna("")
tdfid_matrix = tfidf.fit_transform(muestra['genres_clean'])
cosine_similarity = linear_kernel(tdfid_matrix, tdfid_matrix)


In [11]:
muestra

,id,title,genres_clean
0,862,Toy Story,"Animation, Comedy, Family"
1,8844,Jumanji,"Adventure, Fantasy, Family"
2,15602,Grumpier Old Men,"Romance, Comedy"
3,31357,Waiting to Exhale,"Comedy, Drama, Romance"
4,11862,Father of the Bride Part II,Comedy
...,...,...,...
23051,89547,Bright Road,Drama
23052,43857,Broadway Melody of 1938,"Music, Romance"
23053,80918,Broadway Rhythm,"Family, Music"
23054,130398,Broadway Serenade,"Romance, Drama, Music"


In [14]:
def recomendacion_movie(id: int):
    if id not in muestra['id'].values:
        return {'mensaje': 'No existe el id de a peicula'}
    
    titulo = muestra.loc[muestra['id'] == id, 'title'].iloc[0]
    idx = muestra[muestra['title'] == titulo].index[0]
    sim_cosine = list(enumerate(cosine_similarity[idx]))
    sim_scores = sorted(sim_cosine, key=lambda x: x[1], reverse=True)
    sim_ind = [i for i, _ in sim_scores[1:6]]
    sim_movies = muestra['title'].iloc[sim_ind].values.tolist()
    return {f'peliculas recomendados para {id}': list(sim_movies)}


In [17]:
print(recomendacion_movie(11862)) #Father of the Bride Part II
print(recomendacion_movie(8844))  #jumanji

{'peliculas recomendados para 11862': ['Bio-Dome', 'Friday', 'Black Sheep', 'Happy Gilmore', 'Happiness Is in the Field']}
{'peliculas recomendados para 8844': ['The Indian in the Cupboard', 'The Wizard of Oz', 'Labyrinth', 'Return to Oz', 'Clash of the Titans']}
